<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
</div>

In [ ]:
from plan import plan_extras; plan_extras("numpy", "dimension 1")

# intro à numpy 

## pourquoi `numpy`

la librairie numpy est **très** utilisée pour les traitements gourmands en calculs, car

* elle est **beaucoup plus efficace** que python *"de base"* pour cet usage,
* même si elle est un tout petit peu moins générale.

la limitation concerne le fait que **les tableaux sont homogènes**.

* en première approximation : tous les éléments ont le même type
* c'est cela qui permet d'avoir un code optimisé (tous les éléments sont alignés, les accès sont directs)
* et en pratique, pour les calculs scientifiques ce n'est en général **pas un problème**.

# pourquoi `matplotlib`

* c'est une librairie de visualisation de données 2D/3D
* au départ visualisations statiques 
  * à la gnuplot, un jeu de données = un graphique à imprimer
* a évolué vers la visualisation interactive

In [ ]:
# ces abréviations sont totalement standard
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
### initialisation matplotlib pour notebook
# préférez utiliser %matplotlib notebook
# mais qui n'est pas utilisable en mode 'slides'
%matplotlib inline
# interactive on : pas besoin d'appeler figure() et show()
plt.ion()

# plan

* dimension 1
  * typage
  * programmation vectorielle
* dimensions supérieures
  * slicing
  * broadcasting
* algèbre linéaire
* types structurés (*structured arrays*)

# dimension 1

créer un objet `np.array` de dimension 1

* par exemple à partir d'une liste
* et beaucoup d'autres méthodes

In [ ]:
powers = np.array( [1, 2, 4, 8, 16, 32])
powers, powers.dtype

### erreur fréquente

attention à bien passer une liste et pas directement les nombres

In [ ]:
try: a = np.array(1, 2, 3, 4)    # pas correct
except Exception as e: print("OOPS", e)


In [ ]:
a = np.array([1,2,3,4]); a # OK

## choisir le type

sans indication de notre part numpy va décider tout seul du type

In [ ]:
# les entrées peuvent être homogènes
foo = np.array([True, False, True])
foo.dtype

In [ ]:
# ou pas
foo = np.array([True, 1, 2.5, 4 + 5j])
foo.dtype

### le type est important !

comme dans des langages plus classiques, on peut perdre de la précision

In [ ]:
# numpy va créer un tableau d'entiers
entiers = np.array([1, 2, 3])
entiers.dtype

In [ ]:
# du coup ATTENTION : perte de précision
entiers[0] = 3.14
entiers

### le type : attribut `dtype`

on peut spécifier un type à la création

In [ ]:
# en spécifiant le paramètre dtype
squares = np.array( [1, 2, 4, 9, 16], dtype=np.float64)
squares, squares.dtype

In [ ]:
squares[0] = 1.5
squares

## types disponibles

[voyez la liste complète https://docs.scipy.org/doc/numpy/user/basics.types.html](https://docs.scipy.org/doc/numpy/user/basics.types.html) 

ce qu'il faut en retenir:

* bool, int, uint, float et complex
* de tailles diverses pour vous permettre d'optimiser la mémoire réellement utilisée
* ces types existent en tant que tel (hors de tableaux)

In [ ]:
# un entier qui vaut 1 sur 1 seul octet, c'est possible !
np_1 = np.int8(1)
# l'équivalent en python natif
py_1 = 1
# il y a bien égalité
np_1 == py_1

In [ ]:
# mais ce ne sont bien entendu pas les mêmes objets
np_1 is py_1

## création/initialisation

## initialisation - constantes

In [ ]:
np.zeros(10)

In [ ]:
np.ones(10)

bien plus efficace que du python 'pur'

In [ ]:
# pur python
%timeit [ 0. for i in range(10**7)]

In [ ]:
# numpy
%timeit np.zeros(10**7)

In [ ]:
# pur python
from sys import getsizeof
getsizeof([ 0. for i in range(10**7)])

In [ ]:
# numpy
getsizeof(np.zeros(10**7, dtype=np.int8))

## non-initialisé

* [`np.empty`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.empty.html) permet d'allouer sans initialiser

In [ ]:
%timeit np.empty(10**7, dtype='float64')

In [ ]:
%timeit np.zeros(10**7, dtype='float64')

### voir aussi `numpy.fill()`

## intervalle - `arange`

[`np.arange`]() ressemble à `range` de python

In [ ]:
np.arange(10, 20, 2)

sauf qu'elle accepte des arguments flottants

In [ ]:
np.arange(1.0, 2.5, 0.5)

et aussi donc un argument `dtype`:

In [ ]:
np.arange(1, 10, dtype=np.complex128)

## intervalle - `linspace`

* [`np.linspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html#numpy.linspace) 
* pour *remplir* un intervalle 
* avec un nombre de points spécifiés

In [ ]:
np.linspace(0., 2., 5)

**très très** utile pour modéliser une fonction sur un intervalle

In [ ]:
# 100 points pour remplir l'intervalle [0..6pi]
X = np.linspace(0., 6 * np.pi, 100)
# appliquer la fonction sin sur tous ces points
Y = np.sin(X)

In [ ]:
plt.plot(X, Y);

### comparaison `arange` et `linspace`

| `arange` | `linspace` |
|--------|----------|
| début (inclus) | début (inclus) |
| fin(exclus) | fin (inclus par défaut) |
| **espace** entre points | **nombre** de points |

## intervalle `logspace` 

* [`np.logspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.logspace.html#numpy.logspace) parfois intéressant 
* pour des échelles **logarithmiques**

In [ ]:
np.logspace(base=2, start=4, stop=10, num=7)

In [ ]:
# la base peut être un complexe si on veut
# ici je prends i**0, i**1, i**2, i**3
np.logspace(start=0, stop=4, base=1j, num=4, endpoint=False)

### progression géométrique : `geomspace`

Je signale rapidement avec `geomspace` la possibilité de créer un échantillonage un peu similaire à `linspace` mais selon cette fois une progression géométrique. Il faut bien sûr prendre des bornes non nulles et de même signe :

In [ ]:
X = np.geomspace(start=1/10, stop=10, num=11)

# programmation vectorielle

* appliquer une fonction sur tous les éléments d'un tableau
  * on l'a vu déjà à l'instant avec `np.sin`
  * toutes fonctions mathématiques usuelles supportées
* ou combiner deux tableaux
  * par exemple les ajouter
* **sans** faire de **boucle** explicite

## programmation vectorielle - plan

1. 1 fonction sur un tableau 
1. 1 opérateur entre 1 tableau et un scalaire
1. un opérateur entre 2 tableaux
1. cas particulier des opérations logiques

===

* le cas 2. (tableau x scalaire) peut être vu
  * comme un cas particulier de 3. (tableau x tableau)
* c'est un cas particulier du broadcasting
  * que l'on étudiera en détail plus loin

## fonction sur un tableau

In [ ]:
# pour appliquer une fonction sur tout un tableau
X = np.linspace(-2., 2.)
Y = np.exp(X)
plt.plot(X, Y);

## fonction sur un tableau

In [ ]:
# on peut préciser la sortie, c'est-à-dire 
# ranger les résultats dans un tableau déjà alloué
X = np.linspace(-2, 2., num=10)
# si on dispose déjà d'un tableau de la bonne taille
Y = np.empty(10)
np.exp(X, out=Y)
plt.plot(X, Y);

## fonction sur un tableau

In [ ]:
# en particulier on peut récrire dans X 
np.exp(X, out=X);

In [ ]:
# et donc ici j'ai X == Y
X == Y

In [ ]:
np.all(X == Y)

* très utile pour optimiser votre code
* ordres de grandeur plus rapide
* si vous faites plusieurs opérations à la suite
* c'est utile de créer un nouveau tableau
* mais seulement un!

## scalaire & tableau

* De manière identique, on peut
* faire toute opération avec une constante

In [ ]:
X = np.linspace(0., 10., 3); X

In [ ]:
3 * (X + 1) ** 2

### exercice

* dessiner un cercle avec matplotlib
* utiliser `plt.plot`

In [ ]:
R = 10
teta = np.linspace(0, 2*np.pi)
X = R * np.sin(teta)
Y = R * np.cos(teta)
plt.plot(X, Y);

## opérations binaires

* on peut aussi faire des opérations **binaires**
* donc cette fois sur la base de **deux tableaux**
* mais qui donc doivent avoir des **tailles compatibles**
  * en première appoximation: tailles identiques
  * on y reviendra (broadcasting)
* attention que `*` fait une **multiplication**
  * et pas un produit matriciel

In [ ]:
x = np.logspace(base=2, start=0, stop=3, num=4); x

In [ ]:
y = np.logspace(base=2, start=10, stop=7, num=4); y

In [ ]:
x * y

## opérations logiques

* même logique qu'opérations binaires

In [ ]:
a = np.array( (1, 2, 3, 4) )
b = np.arange(1, 5)

In [ ]:
compare = (a == b)
compare

In [ ]:
compare.all()

* avec des tableaux de dimension 1
* on a envie d'utiliser `all` et `any` (standard python)
* pour synthétiser une seule valeur booléenne

In [ ]:
all(a==b)

In [ ]:
a[0] = 10
# ne sont plus égaux
all(a==b)

In [ ]:
# mais par contre il reste au moins une égalité
any(a==b)

#### dimensions supérieures

* pour comparer des tableaux de dimensions supérieures
* les fonctions natives python comme `all` et `any` ne fonctionnent pas toujours
* il est préférable d'utiliser les **méthodes** `all` et `any`

```
try:                   print("all(a==b)", all(a==b))
except Exception as e: print("OOPS", e)

OOPS The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
```

### `np.where`

* Généralisation du [`if` expression conditionnelle](https://docs.python.org/3/reference/expressions.html#conditional-expressions)
* [`np.where(condition, si_vrai, si_faux`)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)

### exercice

Soit la fonction à un élément

$ f(x) =\begin{cases}x^2 & x <=1 \\2x-1 & x>1\end{cases} $

In [ ]:
def f(x):
   return x**2 if x <= 1 else 2*x - 1

on peut utiliser `matplotlib` avec des tableaux python natifs

In [ ]:
# en python natif
py_x = range(-5, 6)
py_y = [ f(x) for x in py_x ]

In [ ]:
plt.plot(py_x, py_y);

* on vous demande d'implémenter une fonction identique
* mais qui accepte en entrée un tableau numpy

In [ ]:
def f(x):
   return x**2 if x <= 1 else 2 * x - 1

def fnp(x):
    return np.where(x<=1, x**2, 2*x-1)

In [ ]:
# pour vérifier votre résultat
x = np.linspace(-5, 5, 100)
y = fnp(x)
plt.plot(x, y);

**remarque**

* pour résoudre cette classe de problèmes
* on trouve parfois un style de programmation à base de masque

In [ ]:
def fnp_mask(x):
    mask = x <= 1
    return mask * x**2 + (1-mask) * (x+1)

sinon les opérateurs logiques fonctionnent *presque* comme d'habitude

In [ ]:
# on peut faire ceci
x = np.arange(-2, 3)
-1 <= x

In [ ]:
# Mais pas ça
try: -1 <= x <= 1
except Exception as e: print("OOPS", e)

In [ ]:
# il faut alors utiliser des opérateurs spécialisés
np.logical_and(-1 <= x, x <= 1)